In [ ]:
### If on Ubuntu:
# % sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev

In [1]:
# %pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoModel, AutoTokenizer

model_name = "BAAI/bge-large-zh-v1.5"
local_model_path = "./local_bge_large_zh"

# Download the model and tokenizer
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save the model and tokenizer to local directory
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)


In [43]:
import pdftotext
from langchain_community.document_loaders import TextLoader
from langchain.schema.document import Document

# modified from https://stackoverflow.com/questions/77045559/langchain-load-with-string
def get_text_chunks_langchain(text):
    """ Turns raw string into docs that conform with docs = loader.load()"""
    text_splitter = CharacterTextSplitter(
		separator="\n",
		chunk_size=1000,
		chunk_overlap=50,
		length_function=len
	)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

def load_pdf(filepath):
    """ From a pdf, return a docs"""
    text = ""
    with open(filepath, "rb") as f:
        pdf = pdftotext.PDF(f)
        for page in pdf:
            text += page
    return get_text_chunks_langchain(text)


""" Unit test """
pdf_files = ["./data/CMA_ES.pdf"]
documents = []

for pdf_file in pdf_files:
    docs = load_pdf(pdf_file)

In [30]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-zh-v1.5")

# Create the Chroma vector store
vector_store = Chroma(embedding_function=embeddings)

No sentence-transformers model found with name BAAI/bge-large-zh-v1.5. Creating a new one with MEAN pooling.
/home/nore/miniconda3/envs/mindspore/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like BAAI/bge-large-zh-v1.5 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [2]:
import os
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import TextLoader

bge_embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-zh-v1.5")

No sentence-transformers model found with name BAAI/bge-small-zh-v1.5. Creating a new one with MEAN pooling.
/home/nore/miniconda3/envs/mindspore/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like BAAI/bge-small-zh-v1.5 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.